# Notes
Assumptions:

---

Time Constraints:
Nonprofit is trying to generate interest for gala happening around the beggining of the summer, we assume street teams would be out canvassing in the three preceding months. March - Mid June.

Counter Values: Assume 'entries' and 'exits' columns reflect cumulative counts that could only increase as time moved forward. Thus, we removed any rows with negative values in differential columns and values greater than 100000 (Approximately X% of the rows)

Target Metrics:
Did not differentiate between entries and exits for a station, but rather relied on 'total_traffic' to determine which station would have the most foot traffic at a given time.

Steps:
- Read data from turnstile
- Preprocess Data




In [1]:
#Import required packages
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import seaborn as sns

In [2]:
#Reads files in turnstile directory into a files list 
from os import listdir
from os.path import isfile, join
files = [f for f in listdir('./Data/Turnstile_data') if isfile(join('./Data/Turnstile_data', f))]

In [3]:
# Create initial DataFrame
dfs = pd.DataFrame()
for file in files:
    print("Loading {}".format(str("./Data/Turnstile_data/"+file)))
    df = pd.read_csv('./Data/Turnstile_data/' + file)
    dfs = dfs.append(df)    

dfs.info()  
initial_shape = dfs.shape

Loading ./Data/Turnstile_data/turnstile_160305.txt
Loading ./Data/Turnstile_data/turnstile_160312.txt
Loading ./Data/Turnstile_data/turnstile_160319.txt
Loading ./Data/Turnstile_data/turnstile_160326.txt
Loading ./Data/Turnstile_data/turnstile_160402.txt
Loading ./Data/Turnstile_data/turnstile_160409.txt
Loading ./Data/Turnstile_data/turnstile_160416.txt
Loading ./Data/Turnstile_data/turnstile_160423.txt
Loading ./Data/Turnstile_data/turnstile_160430.txt
Loading ./Data/Turnstile_data/turnstile_160507.txt
Loading ./Data/Turnstile_data/turnstile_160514.txt
Loading ./Data/Turnstile_data/turnstile_160521.txt
Loading ./Data/Turnstile_data/turnstile_160528.txt
Loading ./Data/Turnstile_data/turnstile_160604.txt
Loading ./Data/Turnstile_data/turnstile_160611.txt
Loading ./Data/Turnstile_data/turnstile_160618.txt
Loading ./Data/Turnstile_data/turnstile_170304.txt
Loading ./Data/Turnstile_data/turnstile_170311.txt
Loading ./Data/Turnstile_data/turnstile_170318.txt
Loading ./Data/Turnstile_data/t

In [4]:
#Function to preprocess data
def preprocess(df):
    #Standardize column names
    df.columns = df.columns.str.strip()
    
    #Change Time Column to datetime format and round to nearest hour
    df['TIME'] = pd.to_datetime(df.TIME, format="%H:%M:%S").dt.round('H')
    df['TIME'] = pd.to_datetime(df.TIME, format="%H:%M:%S").dt.time
    
    #Standardize dates, replace dates not in format MM/DD/YEAR to NaN and remove those rows
    df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y', errors='coerce')
    df.dropna(inplace=True)
    
    #Create day of week 'DOW' column from data column
    dfs['DOW'] = df['DATE'].dt.weekday_name
       
    #Drop Unncessary columns
    df = df.drop(['C/A','UNIT','LINENAME', 'DIVISION', 'DATE'], axis=1)   
    
    prior_shape = df.shape
    
    # Remove non 'REGULAR' audits from Desc column 
    df.drop(df.DESC != 'REGULAR', inplace = True)
    
    post_shape = df.shape
    desc_rows_removed_perc = (prior_shape[0]-post_shape[0]) / prior_shape[0]  * 100

    print("Percentage of Non Regular Data Removed  = {:08.6f} %".format(desc_rows_removed_perc))
    
    return df

In [5]:
df = preprocess(dfs)

Percentage of Non Regular Data Removed  = 0.001038 %


In [6]:
#Entry and exit data are cumulative, need to adjust to periodic interval counts.
df['DIFFS_ENTRIES'] = df['ENTRIES'].diff()
df['DIFFS_EXIT'] = df['EXITS'].diff()

In [7]:
#Remove turnstile Data in DIFFS_ENTRIES and DIFFS_EXITS that is less than zero and greater than 1e5 (Borderline)
negative_diff_entries = df['DIFFS_ENTRIES'] < 0
df.loc[negative_diff_entries,'DIFFS_ENTRIES'] = np.nan

large_diff_entries =  df['DIFFS_ENTRIES'] > 1e5
df.loc[large_diff_entries,'DIFFS_ENTRIES'] = np.nan

negative_diff_exits = df['DIFFS_EXIT'] < 0
df.loc[negative_diff_exits,'DIFFS_EXIT'] = np.nan

large_diff_exits =  df['DIFFS_EXIT'] > 1e5
df.loc[large_diff_exits,'DIFFS_EXIT'] = np.nan

In [8]:
#Add Column with Total Number of Individuals Entering and Exiting the Station
df['TOTAL_TRAFFIC'] = df['DIFFS_ENTRIES'] + df['DIFFS_EXIT']

In [9]:
#Remove Unused Data Columns
df.drop(['SCP','DESC','ENTRIES', 'EXITS', 'DIFFS_ENTRIES','DIFFS_EXIT'], axis=1, inplace=True)

In [10]:
#Percent of data equal to NaN
nulls = df['TOTAL_TRAFFIC'].isnull()
percent_null = len(df.loc[nulls,'TOTAL_TRAFFIC']) / len(df)
f'{percent_null:.2%} of the data is not useable'

'3.38% of the data is not useable'

In [11]:
#Remove Rows with NaN Values that are unusable
df.dropna(inplace=True)

In [21]:
#Assumption
#1-Remove Time Between Midnight and 7 am
even_am = datetime.time(11,0,0)
midnight = datetime.time(0,0,0)
df = df.loc[(df.TIME >= even_am) | (df.TIME == midnight)]

#df.sort_values(['STATION','TIME', 'DATETIME'], ascending=[True, True, True])
#post_shape = df.shape
#desc_rows_removed_perc = (1379121-post_shape[0]) / 1379121  * 100

#print("Percentage of Data Removed  = {:08.6f} %".format(desc_rows_removed_perc))

In [22]:
df.TIME.unique()

array([datetime.time(15, 0), datetime.time(19, 0), datetime.time(23, 0),
       datetime.time(11, 0), datetime.time(13, 0), datetime.time(17, 0),
       datetime.time(21, 0), datetime.time(12, 0), datetime.time(16, 0),
       datetime.time(20, 0), datetime.time(0, 0), datetime.time(14, 0),
       datetime.time(18, 0), datetime.time(22, 0)], dtype=object)